In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
proceds = pd.read_csv('PROCEDURES_ICD.csv')
d_proceds = pd.read_csv('D_ICD_PROCEDURES.csv')

## Topic Modelling

Topic modelling uses unsupervised ML to identify clusters or groups of similar words within a body of text

LSA: Latent semantic analysis is a statistical technique for extracting and representing the main ideas in a body of text. LSA is based on the principle that words that are close in meaning tend to be used together in context.

Latent Dirichlet analysis is one of the most popular topic modeling methods. It uncovers the hidden structure in a set of observations by looking at the relationships between words in a document and grouping them into topics.

In [6]:
admissions = pd.read_csv('ADMISSIONS.csv')
# get all the individual hadm_id
hadm_ids = admissions['HADM_ID'].unique()

In [7]:
# combine the two tables
df_proceds = pd.merge(proceds, d_proceds, on='ICD9_CODE', how='inner')

In [8]:
df_proceds = df_proceds[['HADM_ID','ICD9_CODE', 'LONG_TITLE']] # extract the columns we need
df_proceds['ICD9_CODE'] = df_proceds['ICD9_CODE'].astype(str) # convert the ICD9_CODE to string
agg_proceds = df_proceds.groupby('HADM_ID')['ICD9_CODE'].apply(list)

In [9]:
unique_procedures = d_proceds['ICD9_CODE'].unique()
num_unique_prods, num_ids = len(unique_procedures), len(hadm_ids)
num_unique_prods, num_ids

(3809, 58976)

In [10]:
# convert to string
unique_procedures = [str(x) for x in unique_procedures]

In [11]:
vector_length = num_unique_prods  # Assuming IDs start from 0 and are contiguous
binary_tensor = np.zeros((num_ids, vector_length), dtype=int)
d = pd.DataFrame(binary_tensor, columns=unique_procedures)
d.index = hadm_ids
# change name of index column
d.index.name = 'HADM_ID'
d.reset_index(inplace=True)
d

,HADM_ID,851,852,859,861,862,863,864,869,870,...,9233,9239,9241,9957,9958,9959,9960,9961,9962,9963
0,165315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,152223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,124321,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,161859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,129635,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,191113,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58972,101071,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58973,122631,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58974,170407,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
agg_proceds = agg_proceds.explode().reset_index()

In [13]:
# populating the OHV, this takes a while
for i in range(len(agg_proceds)):
    hadm = agg_proceds.loc[i]['HADM_ID']
    icd9 = agg_proceds.loc[i]['ICD9_CODE']
    d.loc[d['HADM_ID'] == hadm, icd9] = 1
    

In [35]:
# find HADM_IDs that have diagnosis with 5990
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
d_diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv')

# combine the two tables, based on ICD9_CODE, without short title
df_diag = pd.merge(diagnoses, d_diagnoses, on='ICD9_CODE', how='left')
df_diag = df_diag.dropna(subset=['ICD9_CODE'])
df_diag['ICD9_CODE'] = df_diag['ICD9_CODE'].astype(str)
df_diag = df_diag[['HADM_ID','ICD9_CODE', 'LONG_TITLE']]

In [37]:
# check if the diagnosis is 5990
df_diag['UTI'] = df_diag['ICD9_CODE'].apply(lambda x: 1 if x == '5990' else 0)
# get the hadm_id that has diagnosis 5990
hadm_ids_5990 = df_diag[df_diag['UTI'] == 1]['HADM_ID'].unique()
hadm_ids_5990

array([114585, 134369, 191817, ..., 122472, 109999, 161999])

In [42]:
d

,HADM_ID,851,852,859,861,862,863,864,869,870,...,9239,9241,9957,9958,9959,9960,9961,9962,9963,UTI
0,165315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,152223,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,124321,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,161859,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,129635,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,191113,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58972,101071,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58973,122631,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58974,170407,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [116]:
from sklearn.model_selection import train_test_split

# split the data into train and test
X = d.drop(['HADM_ID', 'UTI'], axis=1)
y = d['UTI']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Autoencoders

We reduce the sparse OHV down to 32 dimensions.

In [161]:
# autoencoder
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model

In [118]:
input_dim = x_train.shape[1]
encoding_dim = 64

input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="relu")(input_layer)

autoencoder = Model(inputs=input_layer, outputs=encoder)

In [119]:
encoded = autoencoder.predict(x_train)

1475/1475 [==============================] - 1s 439us/step


In [147]:
# creating a classifier

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

class_weight_0 = len(y_train) / (2 * np.sum(y_train == 0))
class_weight_1 = len(y_train) / (2 * np.sum(y_train == 1))
class_weights = {0: class_weight_0, 1: class_weight_1}

model = Sequential()
model.add(Dense(64, input_dim=encoding_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = model.fit(encoded, y_train, epochs=100, batch_size=64, class_weight=class_weights)

Epoch 1/100
738/738 [==============================] - 1s 649us/step - loss: 0.6570 - accuracy: 0.5966
Epoch 2/100
738/738 [==============================] - 0s 655us/step - loss: 0.6319 - accuracy: 0.5697
Epoch 3/100
738/738 [==============================] - 0s 670us/step - loss: 0.6261 - accuracy: 0.5765
Epoch 4/100
738/738 [==============================] - 0s 648us/step - loss: 0.6221 - accuracy: 0.5874
Epoch 5/100
738/738 [==============================] - 0s 652us/step - loss: 0.6180 - accuracy: 0.5936
Epoch 6/100
738/738 [==============================] - 0s 644us/step - loss: 0.6158 - accuracy: 0.5882
Epoch 7/100
738/738 [==============================] - 0s 641us/step - loss: 0.6135 - accuracy: 0.5850
Epoch 8/100
738/738 [==============================] - 0s 671us/step - loss: 0.6117 - accuracy: 0.5726
Epoch 9/100
738/738 [==============================] - 0s 640us/step - loss: 0.6108 - accuracy: 0.5820
Epoch 10/100
738/738 [==============================] - 0s 638us/step - l

In [148]:
# test case
encoded_test = autoencoder.predict(x_test)
model.evaluate(encoded_test, y_test)

369/369 [==============================] - 0s 322us/step - loss: 0.5883 - accuracy: 0.6162


[0.5882849097251892, 0.6162258386611938]

In [149]:
y_pred = model.predict(encoded_test)
y_train_pred = model.predict(encoded)

1475/1475 [==============================] - 0s 259us/step


In [150]:
# check the accuracy of the model using a confusion matrix
from sklearn.metrics import confusion_matrix
test_cm = confusion_matrix(y_test, y_pred.round())
train_cm = confusion_matrix(y_train, y_train_pred.round())
test_cm    

array([[6495, 4070],
       [ 457,  774]])

In [151]:
# calculate balanced accuracy
from sklearn.metrics import balanced_accuracy_score
print(f"{balanced_accuracy_score(y_test, y_pred.round()) * 100:.2f}% - Balanced accuracy Test")
print(f"{balanced_accuracy_score(y_train, y_train_pred.round()) * 100:.2f}% - Balanced accuracy Train")

62.18% - Balanced accuracy Test
74.57% - Balanced accuracy Train


In [152]:
# check model history
pd.DataFrame(history.history)

,loss,accuracy
0,0.656999,0.596630
1,0.631889,0.569691
2,0.626065,0.576452
3,0.622065,0.587368
4,0.618019,0.593620
...,...,...
95,0.528838,0.640102
96,0.531220,0.648050
97,0.531708,0.650678
98,0.531453,0.659559


Balanced Accuracy comes to be around 0.61% purely based on encoding for the test set.

## Trying with the Sparse Vector instead of Embedding layer

In [163]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_train_scaled

array([[-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102],
       [-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102],
       [-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102],
       ...,
       [-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102],
       [-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102],
       [-0.0195362 , -0.01381286, -0.00797435, ..., -0.07795773,
        -0.1156782 , -0.01527102]])

In [167]:
model_sparse = Sequential()
model_sparse.add(Dense(64, input_dim=3809, activation='relu'))
model_sparse.add(Dropout(0.2))
model_sparse.add(Dense(32, activation='relu'))
model_sparse.add(Dropout(0.2))
model_sparse.add(Dense(16, activation='relu'))
model_sparse.add(Dropout(0.2))
model_sparse.add(Dense(1, activation='sigmoid'))

model_sparse.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
history_sparse = model_sparse.fit(x_train_scaled, y_train, epochs=100, batch_size=64, class_weight=class_weights)

Epoch 1/100
738/738 [==============================] - 1s 1ms/step - loss: 0.6682 - accuracy: 0.5307
Epoch 2/100
738/738 [==============================] - 1s 1ms/step - loss: 0.6110 - accuracy: 0.5828
Epoch 3/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5864 - accuracy: 0.5983
Epoch 4/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5697 - accuracy: 0.6173
Epoch 5/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.6386
Epoch 6/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.6467
Epoch 7/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5436 - accuracy: 0.6321
Epoch 8/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.6406
Epoch 9/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5256 - accuracy: 0.6521
Epoch 10/100
738/738 [==============================] - 1s 1ms/step - loss: 0.5214 - accura

In [168]:
# test case
x_test_scaled = scaler.transform(x_test)
model_sparse.evaluate(x_test_scaled, y_test)

y_pred_sparse = model_sparse.predict(x_test_scaled)
y_train_pred_sparse = model_sparse.predict(x_train_scaled)

# check the accuracy of the model using a confusion matrix
test_cm_sparse = confusion_matrix(y_test, y_pred_sparse.round())
train_cm_sparse = confusion_matrix(y_train, y_train_pred_sparse.round())

# calculate balanced accuracy
print(f"{balanced_accuracy_score(y_test, y_pred_sparse.round()) * 100:.2f}% - Balanced accuracy Test")
print(f"{balanced_accuracy_score(y_train, y_train_pred_sparse.round()) * 100:.2f}% - Balanced accuracy Train")

1475/1475 [==============================] - 1s 355us/step
59.26% - Balanced accuracy Test
81.75% - Balanced accuracy Train
